## 뉴스 그룹 데이터를 기술/패션/스포츠/정치/사회 등의 카테고리로 분류
-텍스트 분류는 특정 문서의 분류를 학습 데이터를 통해 학습해 모델을 생성한 뒤 이 학습 모델을 이용해 다른 문서의 분류를 예측하는 것이다.

-사이킷런은 fetch_20newsgroups() API를 이용해 뉴스그룹의 분류 수행 예제 데이터를 제공한다.

-텍스트를 처로 벡터화  변환하면 희소행렬 형태가 된는데 이러한 희소행렬을 잘 철ㅣ하는 알고리즘은 로지스틱 회귀, SVM, Naive-Baysian 이 있다. 

-먼저 텍스트 기반 분류는 정규화 후 피처 벡터화를 할것이고 그 후 적합한 머신러닝 알고리즘을 이용해 분류를 학습 /예측/ 평가 할것이다.

### 텍스트 정규화

fetch_20newsgroups는 사이킷런의 다른 데이터 세트 예제와 같이 딕셔너리와 유사한 bunch 객체를 반환한다.

In [1]:
from sklearn.datasets import fetch_20newsgroups
news_data= fetch_20newsgroups(subset='all',random_state=156)

In [2]:
print(news_data.keys())
# filenames는 인터넷에서 내려받아 로컬 컴퓨터에 저장하는 디렉토리와 파일명을 지칭한다

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [3]:
import pandas as pd

# Target 클래스의 구성
print('target 클래스의 값과 분포도 \n',pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n',news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


Target클래스의 값은 0부터 19까지 20개로 구성되어 있다.

개별 텍스트 데이터 확인

In [4]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

- 데이터를 보면 뉴스 그룹 기사의 내용+뉴스그룹 제목, 작성자,소속,이메일(Footer) 등의 다양한 정보를 가지고 있다. 
- 이 중에서 내용을 제외한 다른 정보는 제거한다(다른 푸터 정보들은 뉴스그룹 분류이 Tartget클래스 값과 유사한 데이터를 가진 경우가 많기 때문이다. 
- 또한 fetch_20newsgroups()는 subset 파라미터를 이용해 학습데이터 세트와 테스트 데이터 세트를 분리해 내려받을 수 있다.

In [5]:
from sklearn.datasets import fetch_20newsgroups

#subset='train'으로 학습용 데이터만 추출, remove=('headers','footers','quotes')로 내용만 추출
train_news= fetch_20newsgroups(subset='train',remove=('headers','footers','quotes'),
                  random_state=156)
X_train= train_news.data
y_train= train_news.target

#subset='test'으로 테스트 데이터만 추출, removev=('headers','footers','quotes')로 내용만 추출
test_news= fetch_20newsgroups(subset='test',remove=('headers','footers','quotes'),
                             random_state=156)

X_test= test_news.data
y_test= test_news.target
print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data),len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


### 피처 벡터화 변환과 머신러닝 모델학습/예측 평가
- CountVectorizer를 이용해 학습 데이터의 텍스트를 피처 벡터화하겠다. 
-테스트 데이터 역시 피처 벡터화를 수행하는데, 한가지 유의점은 테스트 데이터에서 CountVectorizer를 적용시에는 반드시 학습데이터를 이용해 fit()이 수행된 CountVectorizer 객체를 이용해 
테스트 데이터를 transform해야 한다는 것이다. 
- 그래야 학습 시 설정된 CountVectorizer의 피처 개수와 테스트 데이터를 CountVectorizer로 변환할 피처 개수가 같아진다. 
- 테스트 데이터의 피처 벡터화는 학습 데이터에 사용된 CountVectorizer 객체 변수인 cnt_vect.transform()을 이용해 변환한다
- 테스트 데이터의 피처 벡터화 시 fit_transform을 사용하면 안되는데, CountVectorizer.fit)transform을 테스트 데이터 세트에 적용하면 테스트 데이터 기반으로 다시 Contvectorizer가 fit 을 수행하고 transform하기 때문에 학습 시 사용된 피처 개수와 예측 시 사용할 피처 개수가 달라진다.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 피처 벡터화 변환 수행
cnt_vect= CountVectorizer()
cnt_vect.fit(X_train) # 데이터 변환을 위한 기준 정보 설정
X_train_cnt_vect= cnt_vect.transform(X_train) # 데이터 변환

# 학습 데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피처 벡터화 변환을 수행한다.
X_test_cnt_vect= cnt_vect.transform(X_test) 

print('학습 데이터 텍스트의 CountVectorizer Shape:',X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape: (11314, 101631)


- CounterVectorizer로 피처 추출한 결과 113141개의 문서에서 피처, 즉 단어가 101631개 만들어짐
- 피처 벡터화된 데이터에 로지스틱 회귀 적용해 분류 예측해보자

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Logistic Regression이용해 학습/예측/평가 수행
lr_clf= LogisticRegression(max_iter=500)
lr_clf.fit(X_train_cnt_vect,y_train)
pred= lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:3f}'.format(accuracy_score(y_test,pred)))

- Count 기반 피처 벡터화된 데이터 세트의 로지스틱 회귀 예측 모델의 정확도는 0.604488
- 이번에는 TF-IDF 기반으로 벡터화를 변경해 예측 모델 수행

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환
tfidf_vect= TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect= tfidf_vect.transform(X_train)
X_test_tfidf_vect= tfidf_vect.transform(X_test)

# LogisticRegression 을 이용해 학습/예측/평가 수행
lr_clf= LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
pred=lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

- TF-IDF 기반 피처 벡터화된 데이터 세트의 로지스틱 회귀 예측 모델의 정확도는 0.674로, Count기반보다 더 높은 정확도를 제공한다

모델 성능 향상
- 모델의 성능 향상 방법은 1. 좋은 모델 선택 2. 최상의 피처 전처리 수행
- 1은 선택했으니 피처 전처리를 위해 위에서 TF-IDF는 기본 파라미터를 적용했지만 더 다양한 파라미터를 적용해보자
- TfidVectorizer 클래스의 스톱워드를 english로 변경하고 nrgram_range는 (1,1)에서 (1,2)로, max_df= 300으로 변경한 뒤 다시 예측 성능 측정

In [ ]:
tfidf_vect= TfidfVectorizer(stop_words="english", ngram_range= (1,2),max_df= 300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect= tfidf_vect.transform(X_train)
X_test_tfidf_vect= tfidf_vect.transform(X_test)

lr_clf= LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
pred= lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

기본 파라미터보다 성능이 0.022%p 증가한 것을 알 수 있다.

#### GridSearchCV를 이용해 로지스틱 회귀의 하이퍼 파라미터 최적화 수행

로지스틱 회귀의 C 파라미터만 변경하며서 최적의 C 값 찾은 뒤 C값으로 학습된 모델에서 테스트 데이터로 예측해 성능 평가

In [ ]:
from sklearn.model_selection import GridSearchCV

# 최적 C값 도출 튜닝 수행. CV는 3 폴드 세트로 설정
params= {'C':[0.01,0.1,1,5,10]}
grid_cv_lr= GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect,y_train)
print('Logistic Regression best C parameter:', grid_cv_lr.best_params_)

# 최적 C 값으로 학습된 grid_cv로 예측 및 정확도 평가
pred= grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

로지스틱 회귀의 C가 10일 대 GridSearchCV의 교차 검증 테스트 세트에서 가장 좋은 예측 성능을 나타냈으며, 이를 테스트 데이터 세트에 적용해 약 0.703으로 조금더 향상된 성능 수치가 나왔다.

### 사이킷런 파이프라인 사용 및 GridSearchCV와의 결합
- 사이킷런의 Pipeline클래스를 이용하면 피처 벡터화와 ML알고맂ㅁ 학습/예측을 위한 코드 작성을 한번에 할 수 있다.
- 파이프라인이란, 데이터의 가공, 변환 등의 전처리와 알고리즘 적용을 수도관처럼 한꺼번에 스트림 기반으로 처리한다는 의미이다.
- 사이킷런의 파이프라인은 텍스트 기반의 피처 벡터화, 모든 데이터 전처리 작업과 Estimatr를 결합할 수 있다.

다음은 앞선 텍스트 분류 예제 코드를 Pipeline을 이용해 다시 작성한 코드이다

In [ ]:
from sklearn.pipeline import Pipeline
pipeline= Pipeline([('tfidf_vect',TfidfVectorizer(stop_words='english')),('lr_clf',LogisticRegression(random_state=156))])

- TfidfVectorizer객체를 tfidf_vect라는 객체 변수 명으로, LogisticRegression객체를 lr_vlf라는 객체 변수명으로 생성한 뒤 이 두 객체를 파이프라인으로 
연결하는 Pipeline을 생성한다는 의미
- 아래 코드를 보면 TfldfVectorizer의 학습 데이터와 테스트 데이터에 대한 fit과 transform수행을 통한 피처 벡터화와 LogisticRegressor의 fit과 predict수행을 통한 머신러닝 모델의
학습과 예측이 Pipeline의 fit과 predict로 통일돼 수행됨을 알 수 있다. 
- 이렇게 파이프라인을 적용하면 머신러닝 코드를 더 직관적이고 쉽게 작성가능하다

In [ ]:
pipeline= Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english',ngram_range=(1,2),max_df=300)),
    ('lr_clf',LogisticRegression(C=10))
])
#별도의 TfidVectorizer 객체의 fit(),transform()과 LogisticRegression의 fit, predict가 필요없음

pipeline.fit(X_train,y_train)
pred= pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

- 파이프라인 기반 하이퍼 파라미터 튜닝 GridSearchCV 방식으로 진행
- 피처 벡터화와 하이퍼파라미터 모두 한번에 GridSearchCV 이용해 최적화 가능

- Grid SearchCV에 Estimator가 아닌 Pipeline을 입력할 경우에는 param_grid의 입력값 설정이 기존과 약간 다르다. 
- 딕셔너리 형태의 key value값을 가지며, value값을 리스트 형태로 입력하는 것은 동일하다. 다만 Key 값을 살펴보면 'tfidf_vect_ngram_range'와 같이 
하이퍼 파라미터명이 객체 변수명과 결합돼 제공된다.
- Pipeline을 GridSearchCV에 인자로 입력하면 GridSearchCV는 Pipeline을 구성하는 피처 벡터화 객체의 파라미터와 
Estimator객체의 하이퍼 파라미터를각각 구별할 수 있어야 하는데 개별 객체 명과 파라미터 명/하이퍼 파라미터 명을 결합해 key값으로 할당한다. 
- 가령 TfdifVectorizer객체 변수인 tfdif_vect의 ngram_range파라미터 값을 변화시키면서 최적화하기를 원한다면 객체 변수명인 tfdif_vect에 
언더바 2개를 연달아 붙인 뒤 파라미터 명인 ngram_range를 결합해 'tfidf_vect_nrgram_range'를 key값으로 할당하는 것.

#### 문제점

Pipeline + GridSearchCV 적용 시 유의점은 모든 파라미터를 최적화하려면 매우 많은 튜닝시간이 소모된다.

In [ ]:
# 해당 코드는 데탑 혹은 므시로 실행하도록 하자
from sklearn.pipeline import Pipeline

pipeline= Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf',LogisticRegression())
])

#Pipeline에 기술된 가각 개체 변수에 언더바 2개를 연달아 붙여 GridSearchCV에 사용됨
# 파라미터, 하이퍼 파라미터 이므과 값을 설정

params= { 'tfidf_vect__ngram_range':[(1,1),(1,2),(1,3)],
        'tfidf_vect__max_df':[100,300,700],
        'lr_clf__C':[1,5,10]}

#GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe= GridSearchCV(pipeline, param_grid= params,cv=3, scoring="accuracy",verbose=1)
grid_cv_pipe.fit(X_train,y_train)
print(grid_cv_pipe.best_params_,grid_cv_pipe.best_score_)

pred= grid_cv_pipe.predict(X_test)
print("Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}.format(accuracy_score(y_test,pred))")

- 돌리는데 시간이 오래걸리는 관계로  
- 예측 정확도는 0.702
-  max_df= 700, ngram_range: (1,2)  0.75552412~
- C하이퍼 파라미터는 10을 적용할 때 가장 좋은 성능이 나왔다.
